In [163]:
from langchain_huggingface import ChatHuggingFace , HuggingFaceEndpoint
from dotenv import load_dotenv

In [164]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
import operator

In [166]:
# llm= HuggingFaceEndpoint(
#     repo_id="google/gemma-2-2b-it",
#     task="text-generation"
# )
# model = ChatHuggingFace(llm=llm)
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
model= ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [167]:
# structured output

class EvalutionSchema(BaseModel):
    feedback:str = Field(description="Detailed Feedback for the Essay"),
    score: int = Field(description="Score out of 10", ge=0 , le=10)

In [168]:
structured_model = model.with_structured_output(EvalutionSchema)

d:\Coding\Self-Coding\langgraph_practice\myenv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Detailed Feedback for the Essay'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [169]:
essay = """
In today’s digital age, artificial intelligence tools like ChatGPT play a significant role in making our daily lives more efficient and productive. ChatGPT acts as a virtual assistant that helps people with learning, problem-solving, and creativity. Students use it to clarify concepts, draft essays, and practice coding. Professionals rely on it to write emails, summarize reports, and even generate fresh ideas for projects.

Beyond work and studies, ChatGPT also supports personal growth. It can recommend books, fitness plans, or cooking recipes, and even provide guidance on communication skills. For those who face language barriers, ChatGPT helps in translation and improving grammar. It also offers companionship by engaging in meaningful conversations, reducing feelings of isolation.

Overall, ChatGPT has become a versatile tool that saves time, improves productivity, and enhances creativity. As it continues to evolve, its role in daily life will only expand, shaping how we learn, work, and connect with the world.


"""

In [174]:
prompt = f"You are an evaluator.Return a JSON output following this format:Now evaluate quality Of the following essay and provide a feedback and assign a score out of 10 \n"
result =structured_model.invoke(prompt)
print(result)

feedback='The essay is well-written and easy to understand. However, it could benefit from a stronger conclusion.' score=8


In [181]:
result.score

8

# Start


In [202]:
class UPSCState(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback:str
    thought_evaluate:str
    overall_feedback:str
    
    individual_scores: Annotated[list[int],operator.add]
    avg_score:float
    

In [203]:
def language_evaluate(state: UPSCState):

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

In [204]:
def analysis_evaluate(state: UPSCState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [205]:
def thought_evaluate(state: UPSCState):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'thought_evaluate': output.feedback, 'individual_scores': [output.score]}

In [206]:
def overall_evaluate(state: UPSCState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["thought_evaluate"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [207]:
# create graph
graph = StateGraph(UPSCState)

#add node

graph.add_node("language_evaluate",language_evaluate)
graph.add_node("analysis_evaluate",analysis_evaluate)
graph.add_node("thought_evaluate",thought_evaluate)
graph.add_node("overall_evaluate",overall_evaluate)


In [208]:
# add edges

graph.add_edge(START,"language_evaluate")
graph.add_edge(START,"analysis_evaluate")
graph.add_edge(START,"thought_evaluate")


graph.add_edge("language_evaluate","overall_evaluate")
graph.add_edge("analysis_evaluate","overall_evaluate")
graph.add_edge("thought_evaluate","overall_evaluate")

graph.add_edge("overall_evaluate",END)

workflow =graph.compile()


In [215]:
essay = """
Technollogy have change the world in manny way. Student now can study from interent and learn without go to libary. But some time it make people lazy becaus they rely only on device not there own brain. Society must find balnce between using technollogy and keeping traditonal learnings.


"""

In [216]:
#execute the graph 
initial_state = {'essay':essay}

final_state = workflow.invoke(initial_state)

print(final_state)

{'essay': '\nTechnollogy have change the world in manny way. Student now can study from interent and learn without go to libary. But some time it make people lazy becaus they rely only on device not there own brain. Society must find balnce between using technollogy and keeping traditonal learnings.\n\n\n', 'language_feedback': "The essay has some good points about technology's impact on learning, but it needs significant improvement in grammar, spelling, and sentence structure.  There are several errors that make it difficult to read and understand. The ideas are present but need clearer expression.  Focus on proofreading and improving sentence construction.", 'analysis_feedback': "The essay presents a superficial analysis of technology's impact on learning.  It identifies a few basic points (internet access for studying, potential for laziness) but lacks depth and supporting evidence. The writing also contains grammatical errors and simplistic sentence structures.  A more in-depth an

In [217]:
final_state['overall_feedback']

"The essay's central idea regarding technology's impact on learning is present but underdeveloped and poorly communicated.  Significant improvements are needed in grammar, spelling, sentence structure, and organization. The analysis is superficial, lacking depth, supporting evidence, and specific examples.  A more structured argument exploring the nuances of technology's influence on various learning styles and the role of educators is required.  Thorough proofreading and rewriting are essential."